# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [13]:
import pandas as pd

#pd.options.mode.chained_assignment = None  # default='warn'

ratings = pd.read_csv('ml-latest-small/ratings.csv', sep = ',')

ratingsCount = ratings.groupby('userId').count().reset_index()
ratingsCount = ratingsCount[['userId','rating']]

ratingsCountMoreHundred = ratingsCount[ratingsCount['rating'] > 100]
ratingsCountMoreHundred.rename(columns = {'rating': 'rating_count'}, inplace = True)

ratingMax = ratings.groupby('userId').max().reset_index()
ratingMaxTimestamp = ratingMax[['userId','timestamp']]
ratingMaxTimestamp.rename(columns = {'timestamp': 'timestamp_max'}, inplace = True)

ratingMin = ratings.groupby('userId').min().reset_index()
ratingMinTimestamp = ratingMin[['userId','timestamp']]
ratingMinTimestamp.rename(columns = {'timestamp': 'timestamp_min'}, inplace = True)

averageTimestamp = ratingsCountMoreHundred\
                   .merge(ratingMaxTimestamp, how='inner')\
                   .merge(ratingMinTimestamp, how='inner')

def average(row):
    return row['timestamp_max'] - row['timestamp_min']

averageTimestamp['timestamp_avg'] = averageTimestamp.apply(average, axis=1)

averageTimestamp.head()

,userId,rating_count,timestamp_max,timestamp_min,timestamp_avg
0,4,204,949982274,949778714,203560
1,8,116,1154474527,1154389340,85187
2,15,1700,1469330735,997937239,471393496
3,17,363,1127476640,1127468587,8053
4,19,423,855195373,855190091,5282


# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [9]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

merged = rzd.merge(auto, how = 'outer', on = 'client_id').merge(air, how = 'outer', on = 'client_id')

merged.head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [10]:
merged.merge(client_base, how = 'outer', on = 'client_id').head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [12]:
# объединение визитов и координат
# координаты могут говорить о спросе на товар в данной местности

visit = pd.DataFrame(
    {
        'user_id': [1, 2, 3, 3, 3, 1],
        'visit_id': [11, 22, 33, 333, 3333, 11],
    }
)

lat_lng = pd.DataFrame(
    {
        'user_id': [1, 2, 3, 3, 3, 1],
        'lat_lng': ['47.111134, 39.700001', 
                    '47.232234, 39.700002', 
                    '47.211134, 39.700003', 
                    '47.223454, 39.700004',
                    '47.235374, 39.700004',
                    '47.232234, 39.700002'],
    }
)

merged = visit.merge(lat_lng, how = 'left', on = 'user_id')

merged.head()

,user_id,visit_id,lat_lng
0,1,11,"47.111134, 39.700001"
1,1,11,"47.232234, 39.700002"
2,2,22,"47.232234, 39.700002"
3,3,33,"47.211134, 39.700003"
4,3,33,"47.223454, 39.700004"
